In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

/Users/markhendricks/Projects/finm-portfolio-2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,48833086,566341992448,The Trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,11332778,57923706880,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,23583292,17685696512,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,9989,248054672,The fund generally will invest at least 90% of...
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,66176,574961600,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,31335696,16903942144,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,816794,1496658176,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,3310338,4560725504,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,7913316,31752601600,The underlying index measures the performance ...


In [5]:
STARTDATE = '2011-01-01'
ENDDATE = '2024-09-30'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Adj Close']

[*********************100%***********************]  12 of 12 completed


In [6]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_20939/3831157974.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [7]:
rets

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2011-02-28,0.006617,0.041710,-0.000437,0.035498,0.014354,-0.002084,0.045204,0.040146,0.002583,0.000090,0.034737,0.007208
2011-03-31,0.008427,0.026581,0.062896,-0.023883,0.000424,-0.001546,-0.010935,0.016667,0.005521,0.000173,0.000120,0.011903
2011-04-30,0.048805,0.045559,0.027327,0.056258,0.015976,0.018412,0.046633,0.058671,0.019034,0.000544,0.028961,0.023780
2011-05-31,-0.013546,-0.051724,-0.029400,-0.022061,0.001332,0.025057,0.010134,-0.041565,0.000000,-0.000099,-0.011214,0.002659
2011-06-30,-0.000097,-0.042645,-0.009355,-0.012107,-0.005706,-0.005044,-0.031060,-0.042581,-0.010776,0.000173,-0.016870,0.007549
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,0.014228,-0.002999,0.019517,0.050602,0.016339,0.018001,0.049274,0.039039,0.013434,0.004582,0.050580,0.017771
2024-06-30,-0.009475,-0.001719,0.026180,-0.018267,0.004794,0.012170,0.018764,-0.017107,0.003233,0.004198,0.035280,0.007615
2024-07-31,0.038512,-0.027981,0.008453,0.025916,0.023522,0.028972,0.076248,0.068587,0.011279,0.004599,0.012109,0.017274


In [8]:
retsx

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.006527,0.041620,-0.000527,0.035408,0.014264,-0.002174,0.045113,0.040056,0.002493,0.034647,0.007118
2011-03-31,0.008255,0.026409,0.062723,-0.024055,0.000251,-0.001719,-0.011107,0.016495,0.005348,-0.000052,0.011730
2011-04-30,0.048261,0.045015,0.026783,0.055715,0.015432,0.017868,0.046089,0.058127,0.018490,0.028418,0.023236
2011-05-31,-0.013447,-0.051625,-0.029301,-0.021962,0.001432,0.025156,0.010233,-0.041465,0.000099,-0.011115,0.002758
2011-06-30,-0.000270,-0.042817,-0.009528,-0.012280,-0.005878,-0.005217,-0.031233,-0.042754,-0.010948,-0.017042,0.007377
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,0.009647,-0.007581,0.014935,0.046020,0.011758,0.013420,0.044692,0.034457,0.008852,0.045998,0.013189
2024-06-30,-0.013674,-0.005917,0.021982,-0.022466,0.000595,0.007972,0.014565,-0.021305,-0.000965,0.031082,0.003417
2024-07-31,0.033913,-0.032580,0.003854,0.021317,0.018924,0.024373,0.071649,0.063989,0.006681,0.007510,0.012676


In [9]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')